In [3]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)
testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()
cnn_test = testdata_pixel.reshape(-1,28,28) /255.0
cnn_train = traindata_pixel.reshape(-1,28,28) /255.0
cnn_test = np.expand_dims(cnn_test,axis=3)
cnn_train = np.expand_dims(cnn_train,axis=3)
cnn_label_test = tf.keras.utils.to_categorical(label_test)
cnn_label_train = tf.keras.utils.to_categorical(label_train)

We have now loaded and reshaped the data. The labels were converted into 10-dimensional to be compatible with the output of the softmax activation function and the dimension of the images was increased by one to show that there is only one color channel. 

In [4]:
simple_model = Sequential([
  Conv2D(6, 5, input_shape=(28, 28, 1), activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(16, 5, activation="relu",padding="valid"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(120, 5, activation="relu",padding="valid"),
  Flatten(),
  Dense(84,activation="relu"),
  Dense(10, activation='softmax'),
])
simple_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
simple_model.fit(
  cnn_train,
  cnn_label_train,
  epochs=5,
  validation_data=(cnn_test, cnn_label_test)
)


Epoch 1/5
1875/1875 [==============================] - 327s 173ms/step - loss: 0.5410 - accuracy: 0.8015 - val_loss: 0.3915 - val_accuracy: 0.8554
Epoch 2/5
1875/1875 [==============================] - 23s 13ms/step - loss: 0.3666 - accuracy: 0.8658 - val_loss: 0.3388 - val_accuracy: 0.8767
Epoch 3/5
1875/1875 [==============================] - 31s 16ms/step - loss: 0.3196 - accuracy: 0.8832 - val_loss: 0.3437 - val_accuracy: 0.8756
Epoch 4/5
1875/1875 [==============================] - 54s 29ms/step - loss: 0.2881 - accuracy: 0.8949 - val_loss: 0.2961 - val_accuracy: 0.8913
Epoch 5/5
1875/1875 [==============================] - 52s 28ms/step - loss: 0.2653 - accuracy: 0.9022 - val_loss: 0.2771 - val_accuracy: 0.8958


In [20]:
import sklearn
import pandas as pd
from sklearn.metrics import confusion_matrix
y_pred = simple_model.predict(cnn_test)
y_pred = np.argmax(y_pred,axis=1)
class_names = np.array(["T-shirt / Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"])
conf = sklearn.metrics.confusion_matrix(y_pred, label_test)
conf_df = pd.DataFrame(conf, index=class_names, columns=class_names)
#disable the following line if you want absolute values
conf_df = round(conf_df*100 / conf_df.sum(axis=1),2)
print(conf_df)

313/313 [==============================] - 3s 10ms/step
               T-shirt / Top  Trouser  Pullover  Dress   Coat  Sandal  Shirt   
T-shirt / Top          80.57     0.40      1.66   1.92   0.09    0.10  17.94  \
Trouser                 0.55    98.09      0.12   0.48   0.09    0.10   0.21   
Pullover                0.73     0.10     91.01   0.19   2.36    0.00   3.87   
Dress                   2.29     1.50      1.89  89.44   2.45    0.00   2.69   
Coat                    0.18     0.10     13.14   1.73  80.76    0.00   8.27   
Sandal                  0.00     0.00      0.00   0.00   0.00   94.65   0.00   
Shirt                   6.87     0.00     10.06   2.02   4.90    0.10  73.90   
Sneaker                 0.00     0.00      0.00   0.00   0.00    1.17   0.00   
Bag                     0.46     0.10      0.47   0.19   0.09    0.58   0.54   
Ankle Boot              0.00     0.00      0.00   0.00   0.00    0.58   0.00   

               Sneaker   Bag  Ankle Boot  
T-shirt / Top     0.

The previous code used a modifyed LeNet-5 architecture with max pooling instead of mean pooling and ReLU activation

In [3]:
vgg_model = Sequential([
  Conv2D(32, 3, input_shape=(28, 28, 1), activation="relu",padding="same"),
  Conv2D(32, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Conv2D(16, 3, activation="relu",padding="same"),
  Conv2D(16, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Flatten(),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(256,activation="relu"),
  Dropout(0.15),
  Dense(10, activation='softmax'),
])
vgg_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
vgg_model.fit(
  cnn_train,
  cnn_label_train,
  epochs=15,
  validation_data=(cnn_test, cnn_label_test),
  batch_size=128
)

Epoch 1/15
469/469 [==============================] - 68s 139ms/step - loss: 0.5713 - accuracy: 0.7912 - val_loss: 1.2909 - val_accuracy: 0.5519
Epoch 2/15
469/469 [==============================] - 64s 137ms/step - loss: 0.3583 - accuracy: 0.8678 - val_loss: 0.3758 - val_accuracy: 0.8656
Epoch 3/15
469/469 [==============================] - 64s 137ms/step - loss: 0.3157 - accuracy: 0.8836 - val_loss: 0.2991 - val_accuracy: 0.8881
Epoch 4/15
469/469 [==============================] - 64s 136ms/step - loss: 0.2850 - accuracy: 0.8956 - val_loss: 0.3202 - val_accuracy: 0.8771
Epoch 5/15
469/469 [==============================] - 64s 137ms/step - loss: 0.2622 - accuracy: 0.9049 - val_loss: 0.3682 - val_accuracy: 0.8592
Epoch 6/15
469/469 [==============================] - 63s 134ms/step - loss: 0.2506 - accuracy: 0.9086 - val_loss: 0.3613 - val_accuracy: 0.8742
Epoch 7/15
469/469 [==============================] - 66s 142ms/step - loss: 0.2366 - accuracy: 0.9125 - val_loss: 0.2665 - val_ac

In [1]:
bastis_model = Sequential([
  Conv2D(16, 5, input_shape=(28, 28, 1), activation="relu",padding="same"),
  Conv2D(16, 5, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(32, 3, activation="relu",padding="same"),
  Conv2D(32, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  BatchNormalization(),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  Conv2D(64, 3, activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=1),
  BatchNormalization(),
  Flatten(),
  Dense(256,activation="relu"),
  Dropout(0.1),
  Dense(512,activation="relu"),
  Dropout(0.1),
  Dense(10, activation='softmax'),
])
bastis_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    min_delta=0.01,
    patience=2,
    verbose=0,
    mode='auto',
    restore_best_weights=True,
    start_from_epoch=0
)
history = bastis_model.fit(
  cnn_train,
  cnn_label_train,
  epochs = 10,
  validation_data=(cnn_test, cnn_label_test),
  batch_size=256
)

NameError: name 'Sequential' is not defined